In [9]:
import pandas as pd
from IPython.display import display
import numpy as np

In [11]:
pd.set_option("display.max_rows", None)

In [13]:
df = None

In [15]:
minVotes = None

In [17]:
class netflixRecommenderSystem:
    def __init__(self, datasetPath, genre, votesThrs, topN):
        self._datasetPath = datasetPath
        self._genre = genre
        self._votesThrs = votesThrs
        self._topN = topN
        self.df = None
        self._minVotes = minVotes

        self.colDrop = ['releaseYear', 'type', 'imdbId', 'availableCountries']
        self.colTitle = {"votes": "imdbNumVotes", "genres": "genres", "title": "title", "rating": "imdbAverageRating"}

    def readFile(self):
        self.df = pd.read_csv(self._datasetPath)

    def cleanDataset(self):
        
        for item in self.colDrop:
            self.df = self.df.drop([item], axis=1)

        
        self.df = self.df.drop_duplicates(subset=[self.colTitle["title"]])

        if type(self.df["imdbNumVotes"][0]) == str:
            self.df = self.df[self.colTitle["votes"]] = self.df[self.colTitle["votes"]].str.replace(',','').astype(float)
        
        self.df = self.df.fillna(0)  

    def filterGenre(self):
        self.df = self.df[self.df[self.colTitle["genres"]].str.contains(self._genre, case = False).fillna(False)]

    def threshVotes(self):
        self._minVotes = np.quantile(self.df[self.colTitle["votes"]], self._votesThrs)
        self.df = self.df.drop(self.df[self.df.imdbNumVotes<self._minVotes].index)

    def weightedAvgScore(self):
        weightedAvg = []
        mean = self.df[self.colTitle["rating"]].mean()
        self.df = self.df.reset_index(drop=True)
        
        for i in range (0, len(self.df[self.colTitle["rating"]])):
            res = ( self.df[self.colTitle["votes"]][i]/self.df[self.colTitle["votes"]][i]+ self._minVotes)*self.df[self.colTitle["rating"]][i] + (self._minVotes/(self.df[self.colTitle["votes"]][i]+self._minVotes))*mean
            weightedAvg.append(res)
        self.df['weightedAverage'] = weightedAvg
    
    def sortNscores(self):
        sort = self.df.sort_values('weightedAverage', ascending=False).head(self._topN)
        display(sort)

    def run(self):
        self.readFile()
        self.cleanDataset()
        self.filterGenre()
        self.threshVotes()
        self.weightedAvgScore()
        self.sortNscores()

In [19]:
path = "C:\\Users\\olgaf\\Documents\\IT with Nick\\Python\\Me coding\\Netflix recom system\\data.csv"
best5Comedy = netflixRecommenderSystem(path, "Comedy", 0.8, 20)
best5Comedy.colDrop = ['releaseYear', 'type', 'imdbId', 'availableCountries']
best5Comedy.colTitle = {"votes": "imdbNumVotes", "genres": "genres", "title": "title", "rating": "imdbAverageRating"}
best5Comedy.run()

C:\Users\olgaf\AppData\Local\Temp\ipykernel_15712\609975338.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df = self.df[self.df[self.colTitle["genres"]].str.contains(self._genre, case = False).fillna(False)]


,title,genres,imdbAverageRating,imdbNumVotes,weightedAverage
1327,Rick and Morty,"Adventure, Animation, Comedy",9.1,648724.0,129274.741551
1348,Reply 1988,"Comedy, Drama, Family",9.0,15698.0,127857.138097
1344,Taskmaster,"Comedy, Game-Show",9.0,24510.0,127856.423828
1407,Kota Factory,"Comedy, Drama",9.0,87771.0,127854.920202
1277,The Office,Comedy,9.0,774037.0,127854.119048
1450,Frieren: Beyond Journey's End,"Adventure, Animation, Comedy",8.9,41032.0,126435.098834
1266,Seinfeld,Comedy,8.9,368487.0,126433.645206
1271,Friends,"Comedy, Romance",8.9,1143439.0,126433.481060
1262,Chappelle's Show,"Comedy, Music",8.8,76045.0,125013.839762
1308,Steins;Gate,"Animation, Comedy, Drama",8.8,82573.0,125013.769626
